In [ ]:
import tweepy
import pymongo
from time import sleep
import six

if six.PY2:
    import urlparse
    longtype = six.integer_types[1]
if six.PY3:
    import urllib.parse as urlparse
    longtype = six.integer_types[0]
    
from email.utils import parsedate
from datetime import datetime

In [ ]:
def parse_datetime(string):
        return datetime(*(parsedate(string)[:6]))

In [ ]:
def save_tweets(statuses,current_since_id):
    for status in statuses:
        status['created_at']=parse_datetime(status['created_at'])
        try:
            status['user']['created_at']=parse_datetime(status['user']['created_at'])
        except:
            pass
        tweets.update({'id':status['id']},status,upsert=True)
        current_id = longtype(status['id'])
        if current_id>current_since_id:
            current_since_id = current_id

    if len(statuses)==0:
        sleep(10)

    return current_since_id

In [2]:
query = "earhquake"
geocode = "37.781157,-122.398720,1km"
lang = "en"
tweetsPerQry = 100  # this is the max the API permits
clean_since_id = False

In [3]:
CONSUMER_KEY = "m4pfoQYtQOCahpKEY55dlogg7"
CONSUMER_SECRET = "TdU3rKhwfspQYrSA0GsPW7IooRkbw9opfZ82KZmADsGvnAUL01"
MONGODB_URI = "mongodb://localhost:27017/test_twitter"

In [ ]:
queries_collection = "test_queries"
tweets_collection = "test_tweets"

In [ ]:
client = pymongo.MongoClient(MONGODB_URI)
parsed_dburi = pymongo.uri_parser.parse_uri(MONGODB_URI)
db = client[parsed_dburi['database']]

In [ ]:
queries = db[queries_collection]
tweets = db[tweets_collection]

In [ ]:
queries.create_index([("query",pymongo.ASCENDING),("geocode",pymongo.ASCENDING),("lang",pymongo.ASCENDING)],unique=True)

tweets.create_index([("id", pymongo.DESCENDING),],unique=True)
tweets.create_index([("coordinates.coordinates",pymongo.GEO2D),])

In [4]:
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [19]:
#without since_id
tweetsPerQry = 100  # this is the max the API permits 
clean_since_id = False
result_type = 'mixed'
wait_time = 3.0

In [ ]:
# clean_since_id=False we dont need a clean new since_id
if not clean_since_id:
    current_query = queries.find_one({'query':query,'geocode':geocode,'lang':lang})
else:
    current_query = None

if current_query:
    since_id = current_query['since_id']
else:
    since_id = None 

In [ ]:
while True:

    sleep(wait_time)
    results = api.search(q=query, count=tweetsPerQry, lang=lang,    geocode=geocode, since_id = since_id, result_type = result_type)
    
    refresh_url = results['search_metadata'].get('refresh_url')
    p = urlparse.urlparse(refresh_url)
    
    if since_id:
        current_since_id = longtype(since_id)
    else:
        current_since_id = 0

    new_since_id = save_tweets(results['statuses'],current_since_id)
    next_results = results['search_metadata'].get('next_results')
    
    while next_results:
                p = urlparse.urlparse(next_results)
                next_results_max_id = dict(urlparse.parse_qsl(p.query))['max_id']
                results = perform_query(q=query,geocode=geocode,lang=lang,count=100,since_id=since_id,max_id=next_results_max_id,result_type=result_type)
                next_results = results['search_metadata'].get('next_results')
                new_since_id = save_tweets(results['statuses'],new_since_id)
        
    new_since_id = str(new_since_id)
    queries.update({'query':query,'geocode':geocode,'lang':lang},{"$set":{'since_id':new_since_id}},upsert=True)
    since_id = new_since_id

In [18]:
api.search(q=query, lang = lang, geocode = geocode, count= )

[]